In [5]:
import numpy as np
import os
from PIL import Image

# CIFAR-10 class names to integer labels mapping
label_map = {
    'airplane': 0,
    'automobile': 1,
    'bird': 2,
    'cat': 3,
    'deer': 4,
    'dog': 5,
    'frog': 6,
    'horse': 7,
    'ship': 8,
    'truck': 9
}

def load_images_from_folder(folder):
    images = []
    labels = []
    for label in sorted(os.listdir(folder)):
        label_folder = os.path.join(folder, label)
        if not os.path.isdir(label_folder):
            continue
        for image_file in os.listdir(label_folder):
            image_path = os.path.join(label_folder, image_file)
            with Image.open(image_path) as img:
                img = img.resize((32, 32))
                img_array = np.asarray(img, dtype=np.float32) / 255.0
                images.append(img_array.flatten())
                labels.append(label_map[label])
    images = np.array(images).T
    labels = np.array(labels).reshape(1, -1)
    return images, labels

class Activation:
    @staticmethod
    def relu(z):
        return np.maximum(0, z)

    @staticmethod
    def d_relu(z):
        return np.where(z > 0, 1.0, 0.0)

    @staticmethod
    def sigmoid(z):
        return 1 / (1 + np.exp(-z))

    @staticmethod
    def d_sigmoid(z):
        s = Activation.sigmoid(z)
        return s * (1 - s)

class Parameters:
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(output_size, input_size) * 0.1
        self.biases = np.zeros((output_size, 1))

class Layer:
    def __init__(self, input_size, output_size, activation):
        self.params = Parameters(input_size, output_size)
        self.activation = activation
        self.z = None
        self.a = None
        self.a_prev = None

    def forward(self, a_prev):
        self.a_prev = a_prev
        self.z = np.dot(self.params.weights, a_prev) + self.params.biases
        self.a = getattr(Activation, self.activation)(self.z)
        return self.a

    def backward(self, dA, learning_rate):
        m = self.a_prev.shape[1]
        dZ = dA
        if self.activation == 'relu':
            dZ = dA * Activation.d_relu(self.z)
        elif self.activation == 'sigmoid':
            dZ = dA * Activation.d_sigmoid(self.z)
        dW = np.dot(dZ, self.a_prev.T) / m
        dB = np.sum(dZ, axis=1, keepdims=True) / m
        dA_prev = np.dot(self.params.weights.T, dZ)

        self.params.weights -= learning_rate * dW
        self.params.biases -= learning_rate * dB
        return dA_prev

class DeepNeuralNetwork:
    def __init__(self, layers_dims, learning_rate=0.001, iterations=1000):
        self.layers = [Layer(layers_dims[i], layers_dims[i+1], 'relu') for i in range(len(layers_dims)-2)]
        self.layers.append(Layer(layers_dims[-2], layers_dims[-1], 'sigmoid'))  # Output layer
        self.learning_rate = learning_rate
        self.iterations = iterations

    def forward_propagation(self, X):
        a = X
        for layer in self.layers:
            a = layer.forward(a)
        return a

    def compute_loss(self, y, y_hat):
        epsilon = 1e-7  # Small constant to prevent log(0)
        y_hat_clipped = np.clip(y_hat, epsilon, 1 - epsilon)  # Clip y_hat to avoid log(0)
        m = y.shape[1]
        loss = -np.mean(y * np.log(y_hat_clipped) + (1 - y) * np.log(1 - y_hat_clipped))
        return loss


    def backward_propagation(self, y, y_hat):
        epsilon = 1e-7
        dA = -(y / np.clip(y_hat, epsilon, 1 - epsilon)) + ((1 - y) / np.clip(1 - y_hat, epsilon, 1 - epsilon))
        for layer in reversed(self.layers):
            dA = layer.backward(dA, self.learning_rate)

    def train(self, X, y):
        for i in range(self.iterations):
            y_hat = self.forward_propagation(X)
            loss = self.compute_loss(y, y_hat)
            if i % 100 == 0:
                print(f"Iteration: {i}, Loss: {loss:.4f}")
            self.backward_propagation(y, y_hat)

    def predict(self, X):
        y_hat = self.forward_propagation(X)
        predictions = y_hat > 0.5
        return predictions

    def print_model(self):
        for i, layer in enumerate(self.layers, 1):
            print(f"Layer {i}: {layer.params.weights.shape[0]} neurons, Activation: {layer.activation}")

# Define the architecture and hyperparameters
layers_dims = [3072, 64, 32, 16, 1]  # Example architecture
learning_rate = 0.001
iterations = 1000

# Initialize and train the network
dnn = DeepNeuralNetwork(layers_dims, learning_rate, iterations)
# Assuming X_train and y_train are already loaded
X_train, y_train = load_images_from_folder('data/train')  # Ensure this path is correct
dnn.train(X_train, y_train)
dnn.print_model()


Iteration: 0, Loss: -0.4624
Iteration: 100, Loss: -56.2995
Iteration: 200, Loss: -56.3670
Iteration: 300, Loss: -56.3884
Iteration: 400, Loss: -56.3974
Iteration: 500, Loss: -56.4017
Iteration: 600, Loss: -56.4043
Iteration: 700, Loss: -56.4062
Iteration: 800, Loss: -56.4078
Iteration: 900, Loss: -56.4092
Layer 1: 64 neurons, Activation: relu
Layer 2: 32 neurons, Activation: relu
Layer 3: 16 neurons, Activation: relu
Layer 4: 1 neurons, Activation: sigmoid
